# Naïve Bayes Classifier

Over text corpus

In [1]:
import pandas as pd
import numpy as np
import re

## Global Variables

In [2]:
trainPositive = dict()
trainNegative = dict()

positiveTotal = 0
negativeTotal = 0

pA = 0.0
pNotA = 0.0

alpha = 0

## Read data

In [3]:
df = pd.read_csv('data_reviews.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6918 entries, 0 to 6917
Data columns (total 2 columns):
Sentiment    6918 non-null int64
Review       6918 non-null object
dtypes: int64(1), object(1)
memory usage: 108.2+ KB


,Sentiment,Review
0,1,The Da Vinci Code book is just awesome
1,1,this was the first clive cussler i ve ever rea...
2,1,i liked the Da Vinci Code a lot
3,1,i liked the Da Vinci Code a lot
4,1,I liked the Da Vinci Code but it ultimatly did...


## Get all words

In [4]:
all_words = "".join(map(str,df.iloc[:,1].values))
all_unique_words = set(all_words.split(' '))
vocab_size = len(all_unique_words)
print('Vocab size: %d' % vocab_size)

Vocab size: 2434


In [5]:
def initialize_dicts():
    #Add words to dictionaries
    for word in all_unique_words:
        trainPositive[word] = 0
        trainNegative[word] = 0

## Model

### Process Review

Function that compute the word count per positive/negative reviews.

I.e: This will compute that the word 'good' appears more often in positive reviews than negatives.

In [7]:
def processReview(review,label):
    global positiveTotal
    global negativeTotal
    review = review.split(' ')
    for word in review:
        if(label == 0 and word != ' '):
            trainNegative[word] = trainNegative.get(word,0)+1
            negativeTotal += 1
        elif(label == 1 and word != ' '):
            trainPositive[word] = trainPositive.get(word,0)+1
            positiveTotal += 1

### Conditional review

Computes the conditional probability p(B_i/A_x)

In [8]:
def conditionalReview(review,label):
    result = 1.0
    review = review.split(' ')
    for word in review:
        result *= conditionalWord(word,label)
    return result

### Conditional word

Laplace Smoothing for the words not present in the training set.

Computes the conditional probability p(B_i | A_x) with smoothing

In [9]:
def conditionalWord(word,label):
    global alpha
    if(label == 0):
        return (trainNegative.get(word,0)+alpha)/(float)(negativeTotal+alpha*vocab_size)
    else:
        return (trainPositive.get(word,0)+alpha)/(float)(positiveTotal+alpha*vocab_size)

## Train

In [14]:
def train():
    print('Starting train')
    global pA
    global pNotA
    display_freq = len(df.index)//10
    initialize_dicts()
    total = 0
    numNegative = 0
    for idx,review in df.iterrows():
        if(idx % display_freq == 0):
            print('%d of %d reviews already processed' % (idx,len(df.index)))
        if review.Sentiment == 0:
            numNegative += 1
        total += 1
        processReview(review.Review,review.Sentiment)
    
    
    pA = numNegative/float(total)
    pNotA = (total - numNegative)/float(total)
    
    print('Train finished')

In [15]:
train()

Starting train
0 of 6918 reviews already processed
691 of 6918 reviews already processed
1382 of 6918 reviews already processed
2073 of 6918 reviews already processed
2764 of 6918 reviews already processed
3455 of 6918 reviews already processed
4146 of 6918 reviews already processed
4837 of 6918 reviews already processed
5528 of 6918 reviews already processed
6219 of 6918 reviews already processed
6910 of 6918 reviews already processed
Train finished


## Test

Let's classify some reviews ;)

In [17]:
def classify(review):
    global pA
    global pNotA
    isNegative = pA * conditionalReview(review,0)
    isPositive = pNotA * conditionalReview(review,1)
    if(isNegative > isPositive):
        print('Negative review')
        return 0
    else:
        print('Positive review')
        return 1

In [19]:
classify('The movie was real crap it sucked')

Negative review


0

In [20]:
classify('I really like it it was awesome')

Positive review


1

In [21]:
classify('I dont know if the movie was good or bad')

Negative review


0

In [ ]:
classify('TRY YOURSELF!')